<a href="https://colab.research.google.com/github/cras-lab/Finance1/blob/main/SamSung_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

yfinance 라이브러리 설치

In [1]:
pip install -q yfinance

필요한 모듈 임포트

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.model_selection import train_test_split

삼성전자 주가를 받아온 뒤 'Adj Close'열 이름을 adj_close로 변경

In [3]:
df = yf.download("005935.KS", start="2020-01-01", end="2022-2-15", progress=False)

df = df.loc[:,['Adj Close'] ]
df.rename( columns={'Adj Close':'adj_close'}, inplace=True)

훈련과 테스트 분할 

In [4]:
y_train, y_test = train_test_split( df.adj_close, shuffle=False  )

X_train = np.arange(0, len(y_train) )
X_test = np.arange( len(y_train), len(df.adj_close))
n_features = 1

y_train = np.array( y_train )
y_test = np.array( y_test )

train_series = y_train.reshape( (len(y_train), n_features))
test_series = y_test.reshape( (len(y_test), n_features))


그래프 표시

In [ ]:
fig, ax = plt.subplots( 1,1, figsize=(15,4))
ax.plot( X_train, y_train, lw=3, label="train data")
ax.plot(X_test, y_test, lw=3, label="test data")
ax.legend( loc="lower left")
plt.show()


주가를 0과 1사이의 값으로 표준화(Normalization)한 다음 그래프 표시

In [ ]:
max_price = np.max( train_series )
min_price = np.min( train_series )


print(max_price, min_price)

train_series = (train_series - min_price) / (max_price-min_price)  
test_series = (test_series - min_price) / (max_price-min_price) 

fig, ax = plt.subplots( 1,1, figsize=(15,4))
ax.plot( X_train, train_series, lw=3, label="train data")
ax.plot(X_test, test_series, lw=3, label="test data")
ax.legend( loc="lower left")
plt.show()


TimeseriesGenerator 형태로 변환

In [7]:
from keras.preprocessing.sequence import TimeseriesGenerator
look_back = 20

train_generator = TimeseriesGenerator( train_series, train_series,
                                      length=look_back,
                                      batch_size = 10)


test_generator = TimeseriesGenerator(test_series, test_series,
                                     length= look_back,
                                     batch_size=10)

LSTM 네트워크 구성

In [9]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

n_neurons = 32
model = Sequential()
model.add( LSTM( n_neurons, input_shape=(look_back, n_features)))
model.add( Dense(1))
model.compile( optimizer='adam', loss='mse')

모델을 적합화 한 다음, 예측 생성

In [ ]:
model.fit( train_generator, epochs=50, verbose=0)

test_prediction = model.predict( test_generator )

예측을 그래프로 표시

In [ ]:
x= np.arange( len(df.adj_close) - len(test_prediction ), len(df.adj_close) )
fig, ax= plt.subplots( 1,1, figsize=(15,5))
ax.plot( X_test, y_test, lw=3, c='y', label='test_data')
ax.plot( x, test_prediction*(max_price-min_price)+min_price,\
        lw=3, c='r', linestyle=':', label='predictions')
ax.legend( loc='lower left')
plt.show()


앞의 테스트는 look-back을 보면서 계속 보정해 나가는 문제가 있다. 향후 일정 기간의 주가 예측을 보려면, 테스트 값을 예측한 값으로 대체해 가면서 그 다음 예측을 이어나가야 한다.

In [ ]:
# Extrapolation 
days_to_tell = 30 # 향후 30일간의 주가 흐름을 예측
extrapolation = np.zeros(days_to_tell)
seed_batch    = ((y_test[:look_back] - min_price)/(max_price-min_price)).reshape((1,look_back, n_features))
current_batch = seed_batch

# extrapolate the next days_to_tell values
for i in range(days_to_tell):
    predicted_value = model.predict(current_batch)[0]
    extrapolation[i] = predicted_value
    current_batch = np.append(current_batch[:,1:,:],[[predicted_value]],axis=1)

In [ ]:
x= np.arange( len(df.adj_close) - len(test_prediction ), len(df.adj_close) - len(test_prediction ) + days_to_tell )
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
ax.plot( X_train, y_train, lw=2, label='train_data')
ax.plot(X_test,y_test, lw=3, c='y', label='test data')
ax.plot( x, extrapolation*(max_price-min_price)+min_price,\
        lw=3, c='r', linestyle=':', label='predictions')

ax.legend(loc="lower left")
plt.show();